In [1]:
from PIL import Image, ImageDraw, ImageFont
import os

def image_generator(s, width  = 400, height = 72, font_size = 42):
    output_folder = 'original_images_singles_mono_2'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for text in s:
        img = Image.new("L", (width, height), color=0)
        font = ImageFont.truetype("C:\\Users\\stefa\\OneDrive\\Documents\\UNI\\YEAR 3\\II\\Thesis\\experiments\\droid-sans-mono\\DroidSansMono.ttf", font_size)
        draw = ImageDraw.Draw(img)
        w, h = draw.textsize(text, font=font)
        h += int(h * 0.21)
        draw.text(((width-w)/2, (height-h)/2), text=text, fill='white', font=font)
        
       
        img.save(os.path.join(output_folder, f"{text}.png"))

In [7]:
droid_font_path = "C:\\Users\\stefa\\OneDrive\\Documents\\UNI\\YEAR 3\\II\\Thesis\\experiments\\droid-sans-mono\\DroidSansMono.ttf"
courier_font_path = "C:\\Users\\stefa\\OneDrive\\Documents\\UNI\\YEAR 3\\II\\Thesis\\experiments\\Courier Prime Sans\\Courier Prime Sans.ttf"

In [2]:
def read_words_from_file(file_path):
    words = []

    with open(file_path, 'r') as file:
        for line in file:
            word = line.strip()
            words.append(word)

    return words

In [3]:
image_generator(read_words_from_file('C:\\Users\\stefa\\OneDrive\\Documents\\UNI\\YEAR 3\\II\\Thesis\\PRPC_blp_sampled_spelling.txt'))

C:\Users\stefa\AppData\Local\Temp\ipykernel_18600\3915829978.py:13: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  w, h = draw.textsize(text, font=font)
C:\Users\stefa\AppData\Local\Temp\ipykernel_18600\3915829978.py:13: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  w, h = draw.textsize(text, font=font)
C:\Users\stefa\AppData\Local\Temp\ipykernel_18600\3915829978.py:13: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  w, h = draw.textsize(text, font=font)
C:\Users\stefa\AppData\Local\Temp\ipykernel_18600\3915829978.py:13: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  w, h = draw.textsize(text, font=font)
C:\Users\stefa\AppData\Local\Temp\ipykernel_18600\3915829978.py:

In [4]:
import numpy as np

def img_to_array(folder_path):
    img_dict = {}
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.png'):
            # Open the image using PIL
            img = Image.open(os.path.join(folder_path, file_name))
            
            # Convert the image to a numpy array
            img_array = np.array(img)
            
            # Add the image name (without '.png') and its corresponding numpy array to the dictionary
            img_name = file_name[:-4]  # Remove the '.png' extension
            img_dict[img_name] = img_array
    
    return img_dict

In [5]:
image_dict = img_to_array('original_images_singles_mono_2')

In [6]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def pca_transform(img_dict, n_components, c):
    # Prepare the data for PCA
    data = []
    for img_array in img_dict.values():
        data.append(img_array.flatten())
    data = np.vstack(data)

    # Apply PCA
    pca = PCA(n_components=n_components)
    imgs_pca_reduced = pca.fit_transform(data)
    imgs_pca_recovered = pca.inverse_transform(imgs_pca_reduced)

    reconstructions_folder = 'reconstructions'
    if not os.path.exists(reconstructions_folder):
        os.makedirs(reconstructions_folder)

    # Recreate the images
    for i, (img_name, img_array) in enumerate(img_dict.items()):
        img_recovered_array = imgs_pca_recovered[i, :].reshape(img_array.shape)
        
        # Create a matplotlib plot
        plt.imshow(img_recovered_array, cmap='gray_r')
        plt.axis('off')

        
        # Check if the subfolder exists, if not create it
        folder_path = os.path.join(reconstructions_folder, img_name)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        # Save the image in the subfolder with the name 'n.png', where n is the number of principal components
        if c // 10 == 0: 
            plt.savefig(os.path.join(folder_path, f"00{c}_{n_components}.png"))
            plt.close()
        else: 
            plt.savefig(os.path.join(folder_path, f"0{c}_{n_components}.png"))
            plt.close()


In [9]:
counter = 1
for i in range(4, 66, 2):
    pca_transform(image_dict, i, counter)
    counter += 1
    #print(counter, '----', counter // 10)

In [21]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def pca_transform_v2(img_dict, n_components):
    # Prepare the data for PCA
    data = []
    for img_array in img_dict.values():
        data.append(img_array.flatten())
    data = np.vstack(data)

    # Apply PCA
    pca = PCA(n_components=n_components)
    imgs_pca_reduced = pca.fit_transform(data)
    imgs_pca_recovered = pca.inverse_transform(imgs_pca_reduced)

    return imgs_pca_recovered


In [22]:
transformations = []

for i in range(2, 166, 2):
    transformations.append(pca_transform_v2(image_dict, i))

KeyboardInterrupt: 

In [23]:
def image_recovery(transformations, img_dict):
    # Create a folder called 'recovered_images'
    recovered_folder = 'recovered_images'
    if not os.path.exists(recovered_folder):
        os.makedirs(recovered_folder)

    keys = list(img_dict.keys())

    # Loop through the keys and values in the img_dict
    for i, (img_name, img_array) in enumerate(img_dict.items()):
        # Loop through the transformations
        for j, transformed_data in enumerate(transformations):
            # Reshape the transformed data
            img_recovered_array = transformed_data[i, :].reshape([72, 400])

            # Create a matplotlib plot
            plt.imshow(img_recovered_array, cmap='gray_r')
            plt.axis('off')

            # Check if the subfolder exists, if not create it
            folder_path = os.path.join(recovered_folder, keys[i])
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

            # Save the image in the subfolder with the name '(i+2)*5.png'
            plt.savefig(os.path.join(folder_path, f"{(j + 2) * 2}.png"))
            plt.close()

In [24]:
image_recovery(transformations, image_dict)